In [1]:
# # Colab setup

# from IPython.display import clear_output as clc

# try:
#     from dlroms import *
# except:
#     !pip install git+https://github.com/NicolaRFranco/dlroms.git
#     from dlroms import *

# try:
#     from dlroms_bayesian import *
# except:
#     !pip install git+https://github.com/FraBet29/APSC_project.git
#     from dlroms_bayesian import *

# clc()

In [1]:
# Import libraries

import numpy as np
import os
import torch

from dolfin import *
from dlroms import *
from dlroms.dnns import *
from dlroms.minns import Interpolate
from dlroms_bayesian.expansions import ExpandedLocal
from dlroms_bayesian.utils import *

from IPython.display import clear_output as clc

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
set_seeds(0)

In [3]:
# Domain and mesh definition

mesh_H = fe.unitsquaremesh(100, 100) # fine mesh
V_H = fe.space(mesh_H, 'CG', 1) 
Nh_H = V_H.dim()

mesh_C = fe.unitsquaremesh(50, 50) # coarse mesh
V_C = fe.space(mesh_C, 'CG', 1)
Nh_C = V_C.dim()

clc()

In [4]:
# Load snapshots

N_train_H = 150

path_train_H = os.path.join('snapshots', 'snapshots_train_H_' + str(N_train_H) + '.npz')
data_train_H = np.load(path_train_H)

mu_train_H, u_train_H = data_train_H['mu'].astype(np.float32), data_train_H['u'].astype(np.float32)
mu_train_H, u_train_H = torch.tensor(mu_train_H).to(device), torch.tensor(u_train_H).to(device)

N_train_C = 2400

path_train_C = os.path.join('snapshots', 'snapshots_train_C_' + str(N_train_C) + '.npz')
data_train_C = np.load(path_train_C)

mu_train_C, u_train_C = data_train_C['mu'].astype(np.float32), data_train_C['u'].astype(np.float32)
mu_train_C, u_train_C = torch.tensor(mu_train_C).to(device), torch.tensor(u_train_C).to(device)

path_test = os.path.join('snapshots', 'snapshots_test.npz')
data_test = np.load(path_test)

N_test = data_test['mu'].shape[0]
mu_test, u_test = data_test['mu'].astype(np.float32), data_test['u'].astype(np.float32)
mu_test, u_test = torch.tensor(mu_test).to(device), torch.tensor(u_test).to(device)

In [ ]:
# Traning architecture

m = 16
k = 4

psi_prime = Dense(Nh_C, 4, activation=None)

psi = Dense(4, 100 * m) + \
		Reshape(4 * m, 5, 5) + \
		Deconv2D(7, (4 * m, 2 * m), 1) + \
		Deconv2D(4, (2 * m, m), 2) + \
		Deconv2D(5, (m, 1), 2, activation=None) + \
		Reshape(-1)

phi = Dense(4, 50 * k) + \
		Dense(50 * k, 50 * k) + \
		Dense(50 * k, 4, activation=None)

print("Trainable parameters:")
print("  Encoder:", psi_prime.dof())
print("  Decoder:", psi.dof())
print("  Dense NN:", phi.dof())

In [12]:
# Load the dense NN and the decoder

psi.load_state_dict(torch.load(os.path.join('checkpoints', 'psi_' + str(N_train_C) + '_' + str(N_train_H) + '.pth'), \
								map_location=device, weights_only=True))
phi.load_state_dict(torch.load(os.path.join('checkpoints', 'phi_' + str(N_train_C) + '_' + str(N_train_H) + '.pth'), \
							map_location=device, weights_only=True))

psi.freeze()
phi.freeze()

# Random initialization

In [6]:
chi = Local(V_C, V_H, support=0.1, activation=None)
chi.He()

model = DFNN(phi, psi, chi)

if torch.cuda.is_available():
	model.cuda()

model.train(mu_train_H, u_train_H, ntrain=N_train_H, epochs=40, loss=mse(euclidean), verbose=True)

model.eval()

with torch.no_grad():
	u_train_H_pred = model(mu_train_H)

error_train = mre(euclidean)(u_train_H, u_train_H_pred)
print(f"Relative training error: {100 * torch.mean(error_train):.2f}%")

with torch.no_grad():
	u_test_pred = model(mu_test)

error_test = mre(euclidean)(u_test, u_test_pred)
print(f"Relative test error: {100 * torch.mean(error_test):.2f}%")

# Deterministic initialization

In [6]:
chi = ExpandedLocal(V_C, V_H, support=0.1, activation=None)
chi.deterministic()

model = DFNN(phi, psi, chi)

if torch.cuda.is_available():
	model.cuda()

model.train(mu_train_H, u_train_H, ntrain=N_train_H, epochs=40, loss=mse(euclidean), verbose=True)

model.eval()

with torch.no_grad():
	u_train_H_pred = model(mu_train_H)

error_train = mre(euclidean)(u_train_H, u_train_H_pred)
print(f"Relative training error: {100 * torch.mean(error_train):.2f}%")

with torch.no_grad():
	u_test_pred = model(mu_test)

error_test = mre(euclidean)(u_test, u_test_pred)
print(f"Relative test error: {100 * torch.mean(error_test):.2f}%")

# Interpolation layer

In [ ]:
model = DFNN(phi, psi, Interpolate(mesh_C, V_C, V_H))

if torch.cuda.is_available():
	model.cuda()

model.eval()

with torch.no_grad():
	u_train_H_pred = model(mu_train_H)

error_train = mre(euclidean)(u_train_H, u_train_H_pred)
print(f"Relative training error: {100 * torch.mean(error_train):.2f}%")

with torch.no_grad():
	u_test_pred = model(mu_test)

error_test = mre(euclidean)(u_test, u_test_pred)
print(f"Relative test error: {100 * torch.mean(error_test):.2f}%")